## Camera Calibration

In this script we get the camera parameters and we manage to get rid of the distorsion. All of the values are stored, so can be accesible at any time and use them for our aplication

In [25]:
#!/usr/bin/env python

import cv2
import numpy as np
import os
import glob
import yaml

# workingdir="/home/pi/Desktop/Captures/"
savedir = 'camera_data/'

# Defining the dimensions of checkerboard
CHECKERBOARD = (9,11)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 


# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

# Extracting path of individual image stored in a given directory
images = glob.glob('./images_calibration/prueba/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    
    cv2.imshow('img',img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

h,w = img.shape[:2]

"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Printing  and saving the results
print("Camera matrix : \n")
print(mtx)
np.save(savedir + 'cam_mtx.npy', mtx)

print("Dist : \n")
print(dist)
np.save(savedir + 'dist.npy', dist)

### UNDISTORSION ####

# Refining the camera matrix using parameters obtained by calibration
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

print("Region of Interest: \n")
print(roi)
np.save(savedir+'roi.npy', roi)

print("New Camera Matrix: \n")
#print(newcam_mtx)
np.save(savedir+'newcam_mtx.npy', new_camera_mtx)
print(np.load(savedir+'newcam_mtx.npy'))

inverse_newcam_mtx = np.linalg.inv(new_camera_mtx)
print("Inverse New Camera Matrix: \n")
print(inverse_newcam_mtx)
np.save(savedir+'inverse_newcam_mtx.npy', inverse_newcam_mtx)

# Method 1 to undistort the image
dst = cv2.undistort(img, mtx, dist, None, new_camera_mtx)

# Displaying the undistorted image
cv2.imshow("undistorted image",dst)
cv2.waitKey(0)

# Closing all remaining windows
cv2.destroyAllWindows()

Camera matrix : 

[[1.42828676e+03 0.00000000e+00 6.54918474e+02]
 [0.00000000e+00 1.42905343e+03 3.46292055e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Dist : 

[[-3.31517840e-02  5.18759991e-01 -4.43647499e-04 -5.12981994e-04
  -1.19772978e+00]]
Region of Interest: 

(1, 2, 1279, 716)
New Camera Matrix: 

[[1.43475037e+03 0.00000000e+00 6.54306457e+02]
 [0.00000000e+00 1.42859460e+03 3.46203464e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Inverse New Camera Matrix: 

[[ 6.96985360e-04  0.00000000e+00 -4.56042022e-01]
 [ 0.00000000e+00  6.99988644e-04 -2.42338493e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Writing the parameters in a file

In [ ]:
data = {'camera_matrix': np.asarray(mtx).tolist(),
        'dist_coeff': np.asarray(dist).tolist(),
        'roi': np.asarray(roi).tolist(),
        'new_camera_mtx': np.asarray(new_camera_mtx).tolist(),
        'inverse_newcam_mtx':np.asarray(inverse_newcam_mtx).tolist()
       }

In [ ]:
# transform the matrix and distortion coefficients to writable lists
# and save it to a file
with open("calibration_matrix.yaml", "w") as f:
     yaml.dump(data, f)

## Getting the Perspective Calibration Right

In [26]:
import numpy as np
import cv2
import glob

# parameters 
writeValues = True
display = False
savedir = "camera_data/"

# Get the center of the image cx and cy
def get_image_center(savedir):
        
    # load camera calibration
    newcam_mtx = np.load(savedir+'newcam_mtx.npy')

    # load center points from New Camera matrix
    cx = newcam_mtx[0,2]
    cy = newcam_mtx[1,2]
    fx = newcam_mtx[0,0]
    return cx,cy

# Load parameters from the camera
def load_parameters(savedir, display):
    # load camera calibration
    savedir = "camera_data/"
    cam_mtx = np.load(savedir+'cam_mtx.npy')
    dist = np.load(savedir+'dist.npy')
    roi = np.load(savedir+'roi.npy')
    newcam_mtx = np.load(savedir+'newcam_mtx.npy')
    inverse_newcam_mtx = np.save(savedir+'inverse_newcam_mtx.npy')
    
    if display:
        print ("Camera Matrix :\n {0}".format(cam_mtx))
        print ("Dist Coeffs :\n {0}".format(dist))
        print("Region of Interest :\n {0}".format(roi))
        print("New Camera Matrix :\n {0}".format(newcam_mtx))
        print("Inverse New Camera Matrix :\n {0}".format(inverse_newcam_mtx))
            
    return cam_mtx,dist,roi,newcam_mtx,inverse_newcam_mtx

def save_parameters(savedir,rotation_vector, translation_vector,newcam_mtx):
    
    # Rotation Vector
    np.save(savedir + 'rotation_vector.npy', rotation_vector)
    np.save(savedir + 'translation_vector.npy', translation_vector)
    
    # Rodrigues
    # print("R - rodrigues vecs")
    R_mtx, jac = cv2.Rodrigues(rotation_vector)
    np.save(savedir + 'R_mtx.npy', R_mtx)  
    
    # Extrinsic Matrix
    Rt = np.column_stack((R_mtx,translation_vector))
    # print("R|t - Extrinsic Matrix:\n {0}".format(Rt))
    np.save(savedir + 'Rt.npy', Rt)
    
    # Projection Matrix      
    P_mtx=newcam_mtx.dot(Rt)
    # print("newCamMtx*R|t - Projection Matrix:\n {0}".format(P_mtx))
    np.save(savedir + 'P_mtx.npy', P_mtx)
    
def load_checking_parameters(savedir):
    
    rotation_vector = np.load(savedir+'rotation_vector.npy')
    translation_vector = np.load(savedir+'translation_vector.npy')
    R_mtx = np.load(savedir+'R_mtx.npy')
    Rt = np.load(savedir+'Rt.npy')
    P_mtx = np.load(savedir+'P_mtx.npy')
    inverse_newcam_mtx = np.load(savedir+'inverse_newcam_mtx.npy')
    
    return rotation_vector, translation_vector,R_mtx,Rt,P_mtx,inverse_newcam_mtx

# Calculate the real Z coordinate based on the center of the image
def calculate_z_total_points(world_points, X_center, Y_center):
    
    total_points_used = len(world_points)
    
    for i in range(1,total_points_used):
    # start from 1, given for center Z=d*
    # to center of camera
        wX = world_points[i,0]-X_center
        wY = world_points[i,1]-Y_center
        wd = world_points[i,2]

        d1 = np.sqrt(np.square(wX) + np.square(wY))
        wZ = np.sqrt(np.square(wd) - np.square(d1))
        world_points[i,2] = wZ

    print(world_points)    
    return world_points

# Lets the check the accuracy here : 
# In this script we make sure that the difference and the error are acceptable in our project. 
# If not, maybe we need more calibration images and get more points or better points

def calculate_accuracy(worldPoints,imagePoints):
    s_arr=np.array([0], dtype = np.float32)
    size_points=len(worldPoints)
    s_describe=np.empty((size_points,),dtype = np.float32)
    
    rotation_vector, translation_vector,R_mtx,Rt,P_mtx,inverse_newcam_mtx = load_checking_parameters(savedir)

    for i in range(0,size_points):
        print("=======POINT # " + str(i) +" =========================")
    
        print("Forward: From World Points, Find Image Pixel\n")
        XYZ1 = np.array([[worldPoints[i,0],worldPoints[i,1],worldPoints[i,2],1]], dtype=np.float32)
        XYZ1 = XYZ1.T
        print("---- XYZ1\n")
        print(XYZ1)
        suv1 = P_mtx.dot(XYZ1)
        print("---- suv1\n")
        print(suv1)
        s = suv1[2,0]    
        uv1 = suv1/s
        print("====>> uv1 - Image Points\n")
        print(uv1)
        print("=====>> s - Scaling Factor\n")
        print(s)
        s_arr = np.array([s/total_points_used + s_arr[0]], dtype = np.float32)
        s_describe[i] = s
        if writeValues == True: 
            np.save(savedir+'s_arr.npy', s_arr)

        print("Solve: From Image Pixels, find World Points")

        uv_1 = np.array([[imagePoints[i,0],imagePoints[i,1],1]], dtype=np.float32)
        uv_1 = uv_1.T
        print("=====> uv1\n")
        print(uv_1)
        suv_1 = s * uv_1
        print("---- suv1\n")
        print(suv_1)

        print("Get camera coordinates, multiply by inverse Camera Matrix, subtract tvec1\n")
        xyz_c = inverse_newcam_mtx.dot(suv_1)
        xyz_c = xyz_c-translation_vector
        print("---- xyz_c\n")
        inverse_R_mtx = np.linalg.inv(R_mtx)
        XYZ = inverse_R_mtx.dot(xyz_c)
        print("---- XYZ\n")
        print(XYZ)

        if calculatefromCam == False:
            cXYZ = cameraXYZ.calculate_XYZ(imagePoints[i,0],imagePoints[i,1])
            print("camXYZ")
            print(cXYZ)


    s_mean, s_std = np.mean(s_describe), np.std(s_describe)

    print(">>>>>>>>>>>>>>>>>>>>> S RESULTS\n")
    print("Mean: "+ str(s_mean))
    #print("Average: " + str(s_arr[0]))
    print("Std: " + str(s_std))

    print(">>>>>> S Error by Point\n")

    for i in range(0,total_points_used):
        print("Point "+str(i))
        print("S: " + str(s_describe[i]) + " Mean: " + str(s_mean) + " Error: " + str(s_describe[i]-s_mean))
        
    return s_mean, s_std

In [27]:
import numpy as np
import cv2
import glob
# import camera_realworld_xyz

writeValues = True
calculatefromCam = True
display = False
savedir = "camera_data/"
# cameraXYZ = camera_realworld_xyz.camera_realtimeXYZ()
# test camera calibration against all points, calculating XYZ

# load camera calibration
cam_mtx,dist,roi,newcam_mtx,inverse_newcam_mtx = load_parameters(savedir, display) 

# load center points from New Camera matrix
cx, cy = get_image_center(savedir)
print("cx:{0}".format(cx) + "cy:{0}".format(cy))

# world center + 9 world points

total_points_used = 10

X_center = -1.5
Y_center = 5.5
# Z_center = -85.0
Z_center = 0.0
# world_points = np.array([[X_center,Y_center,Z_center],
#                       [0.0, -22.0, -86.0], 
#                       [0.0, 0.0, -86.0],
#                       [0.0, 22.0, -86.0],  
#                       [15.0, -22.0, -86.0],
#                       [15.0, 0.0, -86.0],
#                       [15.0, 22.0, -86.0],
#                       [-15.0, -22.0, -86.0],
#                       [-15.0, 0.0, -86.0],
#                       [-15.0, 22.0,-86.0]],dtype=np.float32)

world_points = np.array([[X_center,Y_center,Z_center],
                       [0.0, -22.0, 0.0], 
                       [0.0, 0.0, 0],
                       [0.0, 22.0, 0.0],  
                       [15.0, -22.0, 0.0],
                       [15.0, 0.0, 0.0],
                       [15.0, 22.0, 0.0],
                       [-15.0, -22.0, 0.0],
                       [-15.0, 0.0, 0.0],
                       [-15.0, 22.0,0.0]],dtype=np.float32)


# MANUALLY INPUT THE DETECTED IMAGE COORDINATES HERE

# [u,v] center + 9 Image points
image_points=np.array([[cx,cy],
                       [189, 372],
                       [574,362],
                       [950,347],
                       [206,612],
                       [583,603],
                       [955,596],
                       [189,122],
                       [564,107],
                       [937,98]], dtype=np.float32)

# For Real World Points, calculate Z from d*
# world_points = calculate_z_total_points (world_points, X_center, Y_center)
# print(world_points)


# Get rotatio n and translation_vector from the parameters of the camera, given a set of 2D and 3D points
print("solvePNP")
(success, rotation_vector, translation_vector) = cv2.solvePnP(world_points, image_points, newcam_mtx, dist, flags=cv2.SOLVEPNP_ITERATIVE)

if success:
    print("Sucess:", success)
    print ("Rotation Vector:\n {0}".format(rotation_vector))
    print ("Translation Vector:\n {0}".format(translation_vector))
    
    if writeValues: 
        save_parameters(savedir,rotation_vector, translation_vector,newcam_mtx)
    

# Check the accuracy now
mean, std = calculate_accuracy(world_points, image_points)
print("Mean:{0}".format(mean) + "Std:{0}".format(std))

cx:654.3064573733318cy:346.2034642268318
solvePNP
Sucess: True
Rotation Vector:
 [[-2.22032305]
 [-2.15590637]
 [-0.0557095 ]]
Translation Vector:
 [[-4.93313464]
 [ 0.87615145]
 [84.75951117]]
=======POINT # 0 =========================
Forward: From World Points, Find Image Pixel

---- XYZ1

[[-1.5]
 [ 5.5]
 [ 0. ]
 [ 1. ]]
---- suv1

[[56118.38547906]
 [28182.32901757]
 [   84.62769242]]
====>> uv1 - Image Points

[[663.12082812]
 [333.01544934]
 [  1.        ]]
=====>> s - Scaling Factor

84.62769241998058
Solve: From Image Pixels, find World Points
=====> uv1

[[654.30646]
 [346.20346]
 [  1.     ]]
---- suv1

[[55372.445  ]
 [29298.4    ]
 [   84.62769]]
Get camera coordinates, multiply by inverse Camera Matrix, subtract tvec1

---- xyz_c

---- XYZ

[[-0.7358227 ]
 [ 4.95752034]
 [ 0.04885082]]
=======POINT # 1 =========================
Forward: From World Points, Find Image Pixel

---- XYZ1

[[  0.]
 [-22.]
 [  0.]
 [  1.]]
---- suv1

[[16948.01512996]
 [31575.27897815]
 [   84.9

## Reading from the calibration_matrix.yaml

In [ ]:
# The file contains the camera matrix and de dist_coeff in a dictionary - list format
with open(r'calibration_matrix.yaml') as file:
    doc = yaml.full_load(file)
    mtx = doc["camera_matrix"]
    dist = doc["dist_coeff"]

## Practice Example - 3D to 2D Coordinates

## Function from 3D to 2D

In [ ]:
import cv2
import numpy as np

In [29]:
draw = True
image_path = 'camera_position/WIN_20201027_09_48_05_Pro.jpg'
# world_coordinate = (17.51,17.83,-84.253)
world_coordinate = (10.0, 22.0, 0.0)

In [30]:
def from_3d_to_2d(image_path, world_coordinates, draw):
     
    im = cv2.imread(image_path);
    size = im.shape
    
    # load camera calibration
    savedir = "camera_data/"
    dist = np.load(savedir+'dist.npy')
    newcam_mtx = np.load(savedir+'newcam_mtx.npy')
    rotation_vector = np.load(savedir+'rotation_vector.npy')
    translation_vector = np.load(savedir+'translation_vector.npy')
    
    # Expected this format -> np.array([(0.0, 0.0, 30)])
    world_coordinates = np.array([world_coordinates])
    (new_point2D, jacobian) = cv2.projectPoints(world_coordinates, rotation_vector, translation_vector, newcam_mtx, dist)    
    print("New_point2D:", new_point2D)
    
    if draw :
        cv2.circle(im, (int(new_point2D[0][0][0]),int(new_point2D[0][0][1])),5,(255,0,0), -1)
        # Display image
        cv2.imshow("Test", im)
        #cv2.imwrite(image_path, im)
        cv2.waitKey(0)
        cv2.destroyAllWindows()    
    
    return new_point2D

In [31]:
new_point2D = from_3d_to_2d(image_path, world_coordinate, draw)

New_point2D: [[[946.39846014 517.35712615]]]


## Function from to 2D to 3D

In [38]:
def from_2d_to_3d(image_coordinates):
     
    # load camera calibration
    savedir = "camera_data/"
    cam_mtx,dist,roi,newcam_mtx,inverse_newcam_mtx = load_parameters(savedir, display)
    R_mtx = np.load(savedir + 'R_mtx.npy')
    inverse_R_mtx = np.linalg.inv(R_mtx)
    s_arr = np.load(savedir + 's_arr.npy')
    scalingfactor = s_arr[0]
    
    # Expected this format -> np.array([(0.0, 0.0, 30)])
    u,v = image_coordinates
                                          
    #Solve: From Image Pixels, find World Points
    uv_1 = np.array([[u,v,1]], dtype = np.float32)
    uv_1 = uv_1.T
    suv_1 = scalingfactor * uv_1
    xyz_c = inverse_newcam_mtx.dot(suv_1)
    xyz_c = xyz_c - translation_vector
    XYZ = inverse_R_mtx.dot(xyz_c)
    
    return XYZ

In [43]:
# image_coordinates = [946.65573404,517.46556152]
image_coordinates = [190.0,373.0]

In [44]:
new_point3D = from_2d_to_3d(image_coordinates)
print(new_point3D)

[[  0.04497146]
 [-22.50242188]
 [  0.19925647]]


## Getting coordinates from images - clicking

In [49]:
def click_event(event, x, y, flags, params): 
      
    # checking for left mouse clicks 
    if event == cv2.EVENT_LBUTTONDOWN: 
  
        # displaying the coordinates on the Shell 
        print(x, ' ', y)
        images_coordinates = image_coordinates.append([x,y])
        # displaying the coordinates 
        # on the image window 
        font = cv2.FONT_HERSHEY_SIMPLEX 
        cv2.putText(img, str(x) + ',' +
                    str(y), (x,y), font, 
                    0.5, (255, 0, 0), 2) 
        cv2.imshow('image', img) 

In [50]:
import cv2
import numpy as np

image_path = './camera_position/WIN_20201027_09_48_05_Pro.jpg'
image_coordinates = []    
# reading the image 
img = cv2.imread(image_path, 1) 
  
# displaying the image 
cv2.imshow('image', img) 
  
# setting mouse hadler for the image 
# and calling the click_event() function 
cv2.setMouseCallback('image', click_event)
  
# wait for a key to be pressed to exit 
cv2.waitKey(0) 
  
# close the window 
cv2.destroyAllWindows() 
    

194   373
308   460
202   614
193   122
566   108
575   362
580   606
944   594
949   352
933   98
829   174


In [52]:
image_coordinates

[[194, 373],
 [308, 460],
 [202, 614],
 [193, 122],
 [566, 108],
 [575, 362],
 [580, 606],
 [944, 594],
 [949, 352],
 [933, 98],
 [829, 174]]

**SOURCE**
https://www.fdxlabs.com/calculate-x-y-z-real-world-coordinates-from-a-single-camera-using-opencv/

Distancia de la lente al centro : 85 cm - 0.85 m

Coordenadas x,y,z de los puntos:  